# Results and models

#### Setup and load dataset

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
from src.functions import *

def evaluate_model(model, x_t, y_t):

    print("\nwith next waypoint prediction")
    result_eval = model.evaluate(x_t,y_t)
    print("MSE: ",result_eval)
    print("---------------------------------------------------")
    print("with autoregressive generation:")

    pred = generate(model ,x_t, traj_n=traj_n).numpy()
    result_gen = np.average((y_t - pred[:,1:,:])**2)
    print("MSE: ",result_gen)
    print("Trajectory metrics:")
    metrics, metrics_h = compute_metrics(y_t.numpy()[:,:,:],pred[:,1:,:])
    return metrics


### dataset size experiment

In [2]:
mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("latte_100k_lf", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)



loading BERT model... 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done
loading CLIP model... done
DEVICE:  cuda
loading dataset:  latte_100k_lf ...done
raw X: (100000, 953) 	Y: (100000, 160)
filtered X: (96718, 953) 	Y: (96718, 160)
Train X: (67702, 953) 	Y: (67702, 160)
Test  X: (19344, 953) 	Y: (19344, 160)
Val   X: (9672, 953) 	Y: (9672, 160)


In [2]:
from src.TF4D_mult_features import *
model_path = models_folder+"FINAL_dataset_size_aug_fixsteps/"

model_names =  [
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.01-augment:0.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.01-augment:1.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.1-augment:0.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.1-augment:1.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.5-augment:0.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.5-augment:1.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:0.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5"]
models_metrics = {}
for model_name in model_names:

    model_file = model_path+model_name
    param = file_name2dict(model_file,delimiter="-",show=False)
    print("======================================================================================================")
    print("Dataset size: ",str(100.0*float(param["sf"])),"%\t","augmentation: ","ON "if param['augment']==1 else "OFF", "\n")
    model_tag = "size:" + str(100.0*float(param["sf"]))+"_aug:"+ str(param['augment'])

    model = load_model(model_file, delimiter="-")
    metrics = evaluate_model(model, x_t, y_t)
    models_metrics[model_tag] = metrics


Dataset size:  1.0 %	 augmentation:  OFF 

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 0.01, 'augment': 0}
loading weights:  /home/azureuser/data/models/FINAL_dataset_size_aug_fixsteps/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.01-augment:0.h5

with next waypoint prediction
605/605 [==============================] - 16s 23ms/step - loss: 9.2614e-04
MSE:  0.0009380120318382978
---------------------------------------------------
with autoregressive generation:


100%|██████████| 39/39 [07:26<00:00, 11.45s/it]


MSE:  0.026081953403375196
Trajectory metrics:
pcm:	 0.0
dfd:	 0.46488338701590753
area:	 0.0
cl:	 0.0
dtw:	 8.207000090290196
mae:	 0.11063311492115666
mse:	 0.026081953403375113
Dataset size:  1.0 %	 augmentation:  ON  

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 0.01, 'augment': 1}
loading weights:  /home/azureuser/data/models/FINAL_dataset_size_aug_fixsteps/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.01-augment:1.h5

with next waypoint prediction
605/605 [==============================] - 16s 22ms/step - loss: 5.0294e-04
MSE:  0.000510286248754

100%|██████████| 39/39 [07:18<00:00, 11.25s/it]


MSE:  0.014203454044915438
Trajectory metrics:
pcm:	 0.0
dfd:	 0.3573711908340173
area:	 0.0
cl:	 0.0
dtw:	 5.352905033503025
mae:	 0.07590188144170358
mse:	 0.014203454044915383
Dataset size:  10.0 %	 augmentation:  OFF 

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 0.1, 'augment': 0}
loading weights:  /home/azureuser/data/models/FINAL_dataset_size_aug_fixsteps/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.1-augment:0.h5

with next waypoint prediction
605/605 [==============================] - 15s 22ms/step - loss: 1.3069e-04
MSE:  0.00013290416973177

...

MSE:  0.0023629698830414823
Trajectory metrics:
pcm:	 0.0
dfd:	 0.11344960920497135
area:	 0.0
cl:	 0.0
dtw:	 2.463039142605034
mae:	 0.022455827967532375
mse:	 0.0023629698830414827
Dataset size:  100.0 %	 augmentation:  OFF 

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 1.0, 'augment': 0}
loading weights:  /home/azureuser/data/models/FINAL_dataset_size_aug_fixsteps/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:0.h5

with next waypoint prediction
605/605 [==============================] - 15s 22ms/step - loss: 9.5734e-05
MSE:  9.730462625157

100%|██████████| 39/39 [07:16<00:00, 11.20s/it]


MSE:  0.002290856997408395
Trajectory metrics:
pcm:	 0.0
dfd:	 0.11175631639873772
area:	 0.0
cl:	 0.0
dtw:	 2.3930190131558344
mae:	 0.022014576386467935
mse:	 0.002290856997408402
Dataset size:  100.0 %	 augmentation:  ON  

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 1.0, 'augment': 1}
loading weights:  /home/azureuser/data/models/FINAL_dataset_size_aug_fixsteps/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5

with next waypoint prediction
605/605 [==============================] - 15s 22ms/step - loss: 1.1004e-04
MSE:  0.0001114192345

100%|██████████| 39/39 [07:17<00:00, 11.21s/it]


MSE:  0.00234289053695731
Trajectory metrics:
pcm:	 0.0
dfd:	 0.11383532855574399
area:	 0.0
cl:	 0.0
dtw:	 2.469922672372537
mae:	 0.022737672659383878
mse:	 0.0023428905369572905


In [11]:
print("========== aug 1 ===========\n\t",end="")
for m in ["1k","10k","50k","100k"]:
    print(m, end="\t")
print()
for k in ["mse","mae","dtw","dfd"]:
    print(k, end="\t")
    for n,m in models_metrics.items():
        if "aug:1" in n:
            print(m[k], end="\t")
    print()
print("========= aug 0 ==========")
for k in ["mse","mae","dtw","dfd"]:
    print(k, end="\t")
    for n,m in models_metrics.items():
        if "aug:0" in n:
            print(m[k], end="\t")
    print()

========== aug 1 ===========
	1k	10k	50k	100k	
mse	0.014203454044915383	0.002480648421910659	0.0023629698830414827	0.0023428905369572905	
mae	0.07590188144170358	0.023246562985204867	0.022455827967532375	0.022737672659383878	
dtw	5.352905033503025	2.508415924637376	2.463039142605034	2.469922672372537	
dfd	0.3573711908340173	0.1159301913118411	0.11344960920497135	0.11383532855574399	
========= aug 0 ==========
mse	0.026081953403375113	0.0024354813607664204	0.002241024889381613	0.002290856997408402	
mae	0.11063311492115666	0.02347701585593545	0.021051997831146235	0.022014576386467935	
dtw	8.207000090290196	2.470160787666334	2.3282010349645272	2.3930190131558344	
dfd	0.46488338701590753	0.11683638486097525	0.10978990013544507	0.11175631639873772	


### Num of encoders, decoder and model depth

In [3]:
from src.TF4D_mult_features import *
model_path = models_folder+"FINAL_enc_dec_depth/"

model_names =  [
"TF-num_layers_enc:1-num_layers_dec:3-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:1-num_layers_dec:3-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:1-num_layers_dec:5-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:2-num_layers_dec:3-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:2-num_layers_dec:3-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:2-num_layers_dec:5-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
"TF-num_layers_enc:2-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5"
]
models_metrics_enc_dec = {}
for model_name in model_names:

    model_file = model_path+model_name
    param = file_name2dict(model_file,delimiter="-",show=False)
    print("======================================================================================================")
    print("num Encoders: ",param["num_layers_enc"],"\tnum Decoders: ",param["num_layers_dec"],"\tDepth: ",param['d_model'])
    model_tag = "enc:"+str(param["num_layers_enc"])+"-dec:"+str(param["num_layers_dec"])+"-d"+str(param['d_model'])

    model = load_model(model_file, delimiter="-")
    metrics = evaluate_model(model, x_t, y_t)
    models_metrics[model_tag] = metrics

num Encoders:  1 	num Decoders:  3 	Depth:  256
{'num_layers_enc': 1, 'num_layers_dec': 3, 'd_model': 256, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 1.0, 'augment': 1}
loading weights:  /home/azureuser/data/models/ICRA_TF4D_enc_dec_depth/TF-num_layers_enc:1-num_layers_dec:3-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5

with next waypoint prediction
605/605 [==============================] - 9s 12ms/step - loss: 1.4763e-04
MSE:  0.0001494946627644822
---------------------------------------------------
with autoregressive generation:


100%|██████████| 39/39 [04:16<00:00,  6.57s/it]


MSE:  0.002596976841045097
Trajectory metrics:
pcm:	 0.0
dfd:	 0.12208694415308749
area:	 0.0
cl:	 0.0
dtw:	 2.73886212261568
mae:	 0.02640541335892475
mse:	 0.0025969768410450877
num Encoders:  1 	num Decoders:  3 	Depth:  400
{'num_layers_enc': 1, 'num_layers_dec': 3, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 1.0, 'augment': 1}
loading weights:  /home/azureuser/data/models/ICRA_TF4D_enc_dec_depth/TF-num_layers_enc:1-num_layers_dec:3-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5

with next waypoint prediction
605/605 [==============================] - 10s 15ms/step - loss: 1.2021e-04
MSE:  0.00012197512842249125

100%|██████████| 39/39 [04:57<00:00,  7.63s/it]


MSE:  0.0027276886623445857
Trajectory metrics:
pcm:	 0.0
dfd:	 0.12122696939703666
area:	 0.0
cl:	 0.0
dtw:	 2.702918660119998
mae:	 0.02529149711046403
mse:	 0.002727688662344588
num Encoders:  1 	num Decoders:  5 	Depth:  256
{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 256, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 1.0, 'augment': 1}
loading weights:  /home/azureuser/data/models/ICRA_TF4D_enc_dec_depth/TF-num_layers_enc:1-num_layers_dec:5-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5

with next waypoint prediction
605/605 [==============================] - 13s 18ms/step - loss: 1.5300e-04
MSE:  0.0001545393315609544

...

MSE:  0.002568349239366221
Trajectory metrics:
pcm:	 0.0
dfd:	 0.12146609359756542
area:	 0.0
cl:	 0.0
dtw:	 2.7679229428692076
mae:	 0.027026672595851592
mse:	 0.0025683492393662063
num Encoders:  2 	num Decoders:  5 	Depth:  256
{'num_layers_enc': 2, 'num_layers_dec': 5, 'd_model': 256, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 1.0, 'augment': 1}
loading weights:  /home/azureuser/data/models/ICRA_TF4D_enc_dec_depth/TF-num_layers_enc:2-num_layers_dec:5-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5

with next waypoint prediction
605/605 [==============================] - 14s 20ms/step - loss: 1.6047e-04
MSE:  0.00016268411127384

100%|██████████| 39/39 [06:49<00:00, 10.49s/it]


MSE:  0.00282736868302177
Trajectory metrics:
pcm:	 0.0
dfd:	 0.13193735104718837
area:	 0.0
cl:	 0.0
dtw:	 3.0200743898124935
mae:	 0.03050950657869114
mse:	 0.002827368683021769
num Encoders:  2 	num Decoders:  5 	Depth:  400
{'num_layers_enc': 2, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 1.0, 'augment': 1}
loading weights:  /home/azureuser/data/models/ICRA_TF4D_enc_dec_depth/TF-num_layers_enc:2-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5

with next waypoint prediction
605/605 [==============================] - 17s 24ms/step - loss: 9.9642e-05
MSE:  0.00010097025369759649

100%|██████████| 39/39 [08:02<00:00, 12.36s/it]


MSE:  0.0026205959150691696
Trajectory metrics:
pcm:	 0.0
dfd:	 0.1242314966561472
area:	 0.0
cl:	 0.0
dtw:	 2.7840958375592493
mae:	 0.02682884721046181
mse:	 0.0026205959150691523


In [19]:
for k in ["mse","mae","dtw","dfd"]:
    print(k, end="\t")
print()
for n,m in models_metrics.items():
    if "dec" in n:
        print(n, end="\t")
        for k in ["mse","mae","dtw","dfd"]:
            print(m[k], end="\t")
        print()



mse	mae	dtw	dfd	
enc:1-dec:3-d256	0.0025969768410450877	0.02640541335892475	2.73886212261568	0.12208694415308749	
enc:1-dec:3-d400	0.002727688662344588	0.02529149711046403	2.702918660119998	0.12122696939703666	
enc:1-dec:5-d256	0.0027265313037112566	0.02838061532357939	2.913398924493158	0.12806367503062693	
enc:1-dec:5-d400	0.0024518219509689866	0.024537796733149944	2.602604161995969	0.11872049748753903	
enc:2-dec:3-d256	0.0028254162351774585	0.030203137270130145	2.981241223477885	0.13036128186189053	
enc:2-dec:3-d400	0.0025683492393662063	0.027026672595851592	2.7679229428692076	0.12146609359756542	
enc:2-dec:5-d256	0.002827368683021769	0.03050950657869114	3.0200743898124935	0.13193735104718837	
enc:2-dec:5-d400	0.0026205959150691523	0.02682884721046181	2.7840958375592493	0.1242314966561472	


### Naive predictor
Coping initial trajectory (no modifications)

In [78]:
mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("latte_100k_lf", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)

loading BERT model... 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done
loading CLIP model... done
DEVICE:  cuda
loading dataset:  latte_100k_lf ...done
raw X: (100000, 953) 	Y: (100000, 160)
filtered X: (96718, 953) 	Y: (96718, 160)
Train X: (67702, 953) 	Y: (67702, 160)
Test  X: (19344, 953) 	Y: (19344, 160)
Val   X: (9672, 953) 	Y: (9672, 160)


In [85]:
model_tag="naive_predictor"
traj_in = x_t[0][:,MAX_NUM_OBJS+1:,:].numpy()
traj_out = y_t.numpy()
metrics, metrics_h = compute_metrics(traj_out,traj_in)
models_metrics[model_tag] = metrics

19344it [08:34, 37.62it/s]

pcm:	 0.0
dfd:	 0.13866536138405858
area:	 0.0
cl:	 0.0
dtw:	 3.5680424145512766
mae:	 0.027094356069811483
mse:	 0.004370182933815549


### No language

In [9]:

mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=True)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("latte_100k_lf_cliponly", filter_data = True, base_path=data_folder)
X[:,feature_indices] = 0
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)

loading CLIP model... done
DEVICE:  cuda
clip only
loading dataset:  latte_100k_lf_cliponly ...done
raw X: (100000, 697) 	Y: (100000, 160)
filtered X: (96718, 697) 	Y: (96718, 160)
Train X: (67702, 697) 	Y: (67702, 160)
Test  X: (19344, 697) 	Y: (19344, 160)
Val   X: (9672, 697) 	Y: (9672, 160)


In [ ]:
from src.TF4D_mult_features import *
model_path = models_folder+"FINAL_no_language/"

model_name=  "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:537-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5"
model_tag="no_language"
model_file = model_path+model_name
model = load_model(model_file, delimiter="-")
metrics = evaluate_model(model, x_t, y_t)
models_metrics[model_tag] = metrics

In [11]:
pred = generate(model ,x_t, traj_n=traj_n).numpy()
data_pred = np.array(data)[indices_test].copy()
for i,d in enumerate(data_pred):
    d["output_traj"] = pred[i]

mr.save_data(data_pred,data_name="test_no_language_100k_latte_f", base_path=data_folder)

100%|██████████| 39/39 [07:18<00:00, 11.25s/it]


### Decoder Only

In [13]:
mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("latte_100k_lf", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)

loading BERT model... 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done
loading CLIP model... done
DEVICE:  cuda
loading dataset:  latte_100k_lf ...done
raw X: (100000, 953) 	Y: (100000, 160)
filtered X: (96718, 953) 	Y: (96718, 160)
Train X: (67702, 953) 	Y: (67702, 160)
Test  X: (19344, 953) 	Y: (19344, 160)
Val   X: (9672, 953) 	Y: (9672, 160)


In [14]:
from src.TF4D_decoder_only import *
model_path = models_folder+"FINAL_decoder_only/"

model_name=  "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:16-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse.h5"

def evaluate_model_dec_only(model, x_t, y_t):

    print("\nwith next waypoint prediction")
    # result_eval = model.evaluate(x_t,y_t)
    # print("MSE: ",result_eval)
    print("---------------------------------------------------")
    pred = model.predict(x_t)
    print("Trajectory metrics:")
    metrics, metrics_h = compute_metrics(y_t.numpy()[:,:,:],pred[:,:,:])
    return metrics

x_t_deconly = (x_t[0][:,MAX_NUM_OBJS:-1,:],x_t[1],x_t[2])
model_tag="decoder_only"
model_file = model_path+model_name
model = load_model(model_file, delimiter="-")
metrics = evaluate_model_dec_only(model,x_t_deconly, y_t)
models_metrics[model_tag] = metrics

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 16, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': True, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse'}
loading weights:  /home/azureuser/data/models/FINAL_decoder_only/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:16-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse.h5

with next waypoint prediction
---------------------------------------------------
Trajectory metrics:


19344it [09:06, 35.37it/s]

pcm:	 0.0
dfd:	 0.12620982754684149
area:	 0.0
cl:	 0.0
dtw:	 2.290391868346655
mae:	 0.02421539465031163
mse:	 0.0016075996776673247


NameError: name 'models_metrics' is not defined

In [ ]:
pred = model.predict(x_t_deconly)
data_pred = np.array(data)[indices_test].copy()
for i,d in enumerate(data_pred):
    d["output_traj"] = pred[i]

mr.save_data(data_pred,data_name="test_decoder_only_100k_latte_f", base_path=data_folder)

### clip only

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
from src.functions import *

mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=True)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("latte_100k_lf_cliponly", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)

loading CLIP model... done
DEVICE:  cuda
clip only
loading dataset:  latte_100k_lf_cliponly ...done
raw X: (100000, 697) 	Y: (100000, 160)
filtered X: (96718, 697) 	Y: (96718, 160)
Train X: (67702, 697) 	Y: (67702, 160)
Test  X: (19344, 697) 	Y: (19344, 160)
Val   X: (9672, 697) 	Y: (9672, 160)


In [ ]:
from src.TF4D_mult_features import *
model_path = models_folder+"FINAL_clip_only/"

model_name=  "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:537-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5"
model_tag="decoder_only"
model_file = model_path+model_name
model = load_model(model_file, delimiter="-")
metrics = evaluate_model(model, x_t, y_t)
models_metrics[model_tag] = metrics

### Text Only

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
from src.functions import *

mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("latte_100k_lf_textonly", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)

loading BERT model... 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done
loading CLIP model... done
DEVICE:  cuda
loading dataset:  latte_100k_lf_textonly ...done
raw X: (100000, 953) 	Y: (100000, 160)
filtered X: (96718, 953) 	Y: (96718, 160)
Train X: (67702, 953) 	Y: (67702, 160)
Test  X: (19344, 953) 	Y: (19344, 160)
Val   X: (9672, 953) 	Y: (9672, 160)


In [ ]:
from src.TF4D_mult_features import *
model_path = models_folder+"FINAL_text_only/"

model_name=  "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:537-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5"
model_tag="decoder_only"
model_file = model_path+model_name
model = load_model(model_file, delimiter="-")
metrics = evaluate_model(model, x_t, y_t)
models_metrics[model_tag] = metrics

### Forces interactions

In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
from src.functions import *

mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("forces_only_f", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)


def prepare_x(x):
    objs = pad_array(list_to_wp_seq(x[:,obj_poses_indices],d=3),4,axis=-1) # no speed
    trajs = list_to_wp_seq(x[:,traj_indices],d=4)
    #   return np.concatenate([objs,trajs],axis = 1)
    return trajs[:,:-1,:]

test_dataset = tf.data.Dataset.from_tensor_slices((prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)



loading BERT model... 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done
loading CLIP model... done
DEVICE:  cuda
loading dataset:  forces_only_f ...done
raw X: (10000, 953) 	Y: (10000, 160)
filtered X: (10000, 953) 	Y: (10000, 160)
Train X: (7000, 953) 	Y: (7000, 160)
Test  X: (2000, 953) 	Y: (2000, 160)
Val   X: (1000, 953) 	Y: (1000, 160)
{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 16, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': True, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse'}
loading weights:  /home/azureuser/data/models/forces_onl/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:16-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse.h5


In [5]:
from src.TF4D_decoder_only import *

model_path = models_folder+"forces_onl/"
model_name = "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:16-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse.h5"

model_file = model_path+model_name
model = load_model(model_file, delimiter="-")

# metrics = evaluate_model(model, x_t, y_t)
# models_metrics[model_tag] = metrics

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 16, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': True, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse'}
loading weights:  /home/azureuser/data/models/forces_onl/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:16-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse.h5


In [6]:
pred = model.predict(x_t)

2022-09-12 22:48:38.680329: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-09-12 22:48:38.680738: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2593990000 Hz
2022-09-12 22:48:40.706933: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [13]:
pred_d = np.array(data)[indices_test]
for i,d in enumerate(pred_d):
    d["forces"] = pred[i]
# plt.close('all')
# indices = np.random.choice(range(len(indices_test)), 6)
# pred_t = np.transpose(pred[:,:,:2],[0,2,1])

show_data4D(pred_d,plot_output=False,image_loader=mr.image_loader, color_traj=False, change_img_base=["/home/mirmi/Arthur/dataset/","/home/tum/data/image_dataset/"])
# show_data4D(data_sample,plot_output=False)#,image_loader=mr.image_loader)

TypeError: show_data4D() got an unexpected keyword argument 'plot_output'

# Visualize Results

In [20]:
from src.TF4D_mult_features import *
model_path = models_folder+"FINAL_dataset_size_aug_fixsteps/"

model_name = "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.5-augment:1.h5"
model_file = model_path+model_name

model = load_model(model_file, delimiter="-")


{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 0.5, 'augment': 1}
loading weights:  /home/azureuser/data/models/FINAL_dataset_size_aug_fixsteps/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.5-augment:1.h5


In [21]:
pred = generate(model ,x_t, traj_n=traj_n).numpy()
data_pred = np.array(data)[indices_test].copy()
for i,d in enumerate(data_pred):
    d["output_traj"] = pred[i]

mr.save_data(data_pred,data_name="testpred_100k_latte_f", base_path=data_folder)

100%|██████████| 39/39 [07:23<00:00, 11.37s/it]


In [ ]:
%matplotlib inline
# indices = np.random.choice(range(len(indices_test)), 3)
indices = np.arange(3)

data_array = np.array(data)[indices_test[indices]]

%matplotlib inline
show_data4D(data_array, pred=pred[indices,:,:],color_traj=False,labels=["Ground Truth","Predicted"])
